# Train Summary Plots/Anaylisis

In [ ]:
# Common imports.
from pathlib import Path
from collections import defaultdict

%matplotlib widget
import base

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets

import dfaas_env
import dfaas_utils

Select one or more experiments to view.

**WARNING**: If multiple experiments are selected, they must share the same number of training iterations!

In [ ]:
# Get the list of current experiments to be selected by the user.
exp_prefix = Path("/home/emanuele/marl-dfaas/results")
exps = []
for exp in exp_prefix.iterdir():
    if exp.name.startswith("DFAAS"):  # A single experiment.
        exps.append((exp.name, exp))
        continue

    # The experiment is a directory with sub-experiments, add
    # each experiment individually.
    for sub_exp in exp.iterdir():
        exps.append((f"{exp.name}/{sub_exp.name}", sub_exp))
exps = sorted(exps, reverse=True)

# Show the name as the portion of the path after "results",
# but anyway the values are full Path objects.
exp_select = ipywidgets.SelectMultiple(
    options=exps,
    index=[0],
    description="Experiment(s):",
    style={"description_width": "initial"},
    layout=ipywidgets.Layout(width="70%"),
)

ipywidgets.AppLayout(center=exp_select)

## Experiment selection

This section must be run before any of the following sections to load the selected experiments.

In [ ]:
exps_dir = exp_select.value
assert len(exps_dir) > 0, "must select at least one experiment"

# Preload the data (result.json file) for all selected experiments.
raw_exp_data = {}
for exp_dir in exps_dir:
    raw_exp_data[exp_dir] = dfaas_utils.parse_result_file(exp_dir / "result.json")

print("Selected experiments:")
for exp_dir in exps_dir:
    print(f"  - {exp_dir.name}")

## Reward

In [ ]:
# Common functions for average reward data.


def _average_reward_step(iter, agent):
    """Returns the average reward per step for the given iteration and agent."""
    episodes = iter["env_runners"]["episodes_this_iter"]

    tmp = np.empty(episodes, dtype=np.float32)
    for epi_idx in range(episodes):
        tmp[epi_idx] = np.average(
            iter["env_runners"]["hist_stats"]["reward"][epi_idx][agent]
        )

    return np.average(tmp)


def _get_data(iter_data):
    final_data = {}

    for exp_dir, iters in iter_data.items():
        data = {}
        agents = base.get_env(exp_dir).agents

        data["agents"] = agents
        data["iterations"] = len(iters)
        data["episodes"] = iters[0]["env_runners"]["episodes_this_iter"]

        reward_total_avg = {}  # Average total reward per episode.
        reward_step_avg = {}  # Average reward per step.

        reward_total_avg["all"] = np.empty(data["iterations"], dtype=np.float32)
        for agent in data["agents"]:
            reward_total_avg[agent] = np.empty(data["iterations"], dtype=np.float32)
            reward_step_avg[agent] = np.empty(data["iterations"], dtype=np.float32)

        # For each iteration, get the average reward, since there are multiple
        # episodes played in each iteration.
        for iter in iters:
            # Index starts from one in log files, but Python list from zero.
            iter_idx = iter["training_iteration"] - 1

            reward_total_avg["all"][iter_idx] = np.average(
                iter["env_runners"]["hist_stats"]["episode_reward"]
            )

            for agent in data["agents"]:
                reward_total_avg[agent][iter_idx] = np.average(
                    iter["env_runners"]["hist_stats"][f"policy_policy_{agent}_reward"]
                )
                reward_step_avg[agent][iter_idx] = _average_reward_step(iter, agent)

        data["reward_total_avg"] = reward_total_avg
        data["reward_step_avg"] = reward_step_avg

        final_data[exp_dir] = data

    return final_data

Get `data` and `env` variables, used by subsequents plots.

In [ ]:
data = _get_data(raw_exp_data)
env = base.get_env(exps_dir[0])

### Average per episode (all agents)

In [ ]:
fig = plt.figure(layout="constrained")
fig.canvas.header_visible = False
ax = fig.subplots()

# Limits for the y axis, both for total and single step.
bottom, top = env.reward_range
bottom_total = bottom * env.max_steps
top_total = top * env.max_steps

for exp_dir in exps_dir:
    ax.plot(data[exp_dir]["reward_total_avg"]["all"], label=exp_dir.name)

ax.set_ylim(bottom=bottom_total, top=top_total * len(env.agents))
ax.set_title("Average reward per episode (all agents)")
ax.yaxis.set_major_locator(ticker.MultipleLocator(50))

ax.set_ylabel("Reward per episode")

ax.set_xlabel("Iteration")
ax.xaxis.set_major_locator(
    ticker.MultipleLocator(50)
)  # Show x-axis ticks every 50 iterations.

ax.legend(loc="lower center")
ax.grid(axis="both")
ax.set_axisbelow(True)  # By default the axis is over the content.

### Average per episode (single agents)

In [ ]:
# Limits for the y axis, both for total and single step.
bottom, top = env.reward_range
bottom_total = bottom * env.max_steps
top_total = top * env.max_steps

for agent in env.agents:
    fig = plt.figure(layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    for exp_dir in exps_dir:
        ax.plot(data[exp_dir]["reward_total_avg"][agent], label=exp_dir.name)

    ax.set_ylim(bottom=bottom_total, top=top_total)
    ax.set_title(f"Average reward per episode ({agent = })")
    ax.yaxis.set_major_locator(ticker.MultipleLocator(50))

    ax.set_ylabel("Reward per episode")

    ax.set_xlabel("Iteration")
    ax.xaxis.set_major_locator(
        ticker.MultipleLocator(50)
    )  # Show x-axis ticks every 50 iterations.

    ax.legend(loc="lower center")
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.

## Processed requests

In [ ]:
# Common functions for processed requests.


def _get_data_episode(iter_data, epi_idx, env):
    iter_data = iter_data["env_runners"]["hist_stats"]

    data = defaultdict(lambda: defaultdict())
    (
        data["all"]["processed_reqs"],
        data["all"]["input_reqs"],
        data["all"]["processed_forwarded_reqs"],
    ) = (0, 0, 0)
    for agent in env.agents:
        processed_reqs = np.sum(iter_data["processed_local"][epi_idx][agent])
        input_reqs = np.sum(iter_data["observation_input_requests"][epi_idx][agent])
        processed_forward = np.sum(iter_data["processed_local_forward"][epi_idx][agent])

        data[agent]["processed_reqs"] = processed_reqs
        data[agent]["input_reqs"] = input_reqs
        data[agent]["processed_forwarded_reqs"] = processed_forward
        data["all"]["processed_reqs"] += processed_reqs
        data["all"]["input_reqs"] += input_reqs
        data["all"]["processed_forwarded_reqs"] += processed_forward

    return data


def _get_data(iter_data):
    #  is a disctionary with three levels of depth (experiment, iteration, metrics).
    final_data = defaultdict(lambda: defaultdict(lambda: defaultdict()))

    for exp_dir, iters in iter_data.items():
        agents = base.get_env(exp_dir).agents
        iterations = len(iters)

        # Create the portion of the dictionary for this experiment that
        # contains the average values of the metrics for each iteration.
        for agent in ["all"] + env.agents:
            for key in ["input_reqs", "processed_reqs", "processed_forwarded_reqs"]:
                final_data[exp_dir][agent][key] = np.empty(iterations)

        # For each iteration, calculate the metrics for each episode played,
        # then average the values for the number of episodes of that iteration.
        for iter_idx in range(iterations):
            episodes = iters[iter_idx]["env_runners"]["episodes_this_iter"]

            # Create the data dictionary that contains the metrics for each
            # episode in this iteration.
            data = defaultdict(lambda: defaultdict())
            for agent in ["all"] + env.agents:
                for key in ["input_reqs", "processed_reqs", "processed_forwarded_reqs"]:
                    data[agent][key] = np.empty(episodes, dtype=np.int32)

            # Iterate the episodes.
            for epi_idx in range(episodes):
                data_epi = _get_data_episode(iters[iter_idx], epi_idx, env)

                for agent in ["all"] + env.agents:
                    for key in [
                        "input_reqs",
                        "processed_reqs",
                        "processed_forwarded_reqs",
                    ]:
                        data[agent][key][epi_idx] = data_epi[agent][key]

            # Update iteration data.
            for agent in ["all"] + env.agents:
                for key in ["input_reqs", "processed_reqs", "processed_forwarded_reqs"]:
                    final_data[exp_dir][agent][key][iter_idx] = np.average(
                        data[agent][key]
                    )

    return final_data

Get `data` and `env` variables, used by subsequents plots.

In [ ]:
data = _get_data(raw_exp_data)
env = base.get_env(exps_dir[0])

### Average processed requests per episode

#### All agents

In [ ]:
fig = plt.figure(layout="constrained")
fig.canvas.header_visible = False
ax = fig.subplots()

for exp_dir in exps_dir:
    ratios = data[exp_dir]["all"]["processed_reqs"] / data[exp_dir]["all"]["input_reqs"]
    ratios_forwarded = (
        data[exp_dir]["all"]["processed_forwarded_reqs"]
        / data[exp_dir]["all"]["processed_reqs"]
    )

    ax.plot(ratios, label=exp_dir.name)

ax.set_title("Average processed requests per episode (all agents)")

ax.set_ylabel("Requests")
ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
ax.set_ylim(0, 1)

ax.set_xlabel("Iteration")

ax.legend(loc="lower center")
ax.grid(axis="both")
ax.set_axisbelow(True)  # By default the axis is over the content.

#### Single agents

In [ ]:
for agent in env.agents:
    fig = plt.figure(layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    for exp_dir in exps_dir:
        ratios = (
            data[exp_dir][agent]["processed_reqs"] / data[exp_dir][agent]["input_reqs"]
        )
        # ratios_forwarded = data[exp_dir][agent]["processed_forwarded_reqs"] / data[exp_dir]["all"]["processed_reqs"]

        ax.plot(ratios, label=exp_dir.name)

    ax.set_title(f"Average processed requests per episode ({agent = })")

    ax.set_ylabel("Requests")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.set_ylim(0, 1)

    ax.set_xlabel("Iteration")

    ax.legend(loc="lower center")
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.

### Average processed forwarded requests per episode

#### All agents

In [ ]:
fig = plt.figure(layout="constrained")
fig.canvas.header_visible = False
ax = fig.subplots()

for exp_dir in exps_dir:
    ratios_forwarded = (
        data[exp_dir]["all"]["processed_forwarded_reqs"]
        / data[exp_dir]["all"]["processed_reqs"]
    )

    ax.plot(ratios_forwarded, label=exp_dir.name)

ax.set_title("Average processed forwarded requests per episode (all agents)")

ax.set_ylabel("Requests")
ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
ax.set_ylim(0, 1)

ax.set_xlabel("Iteration")

ax.legend(loc="lower center")
ax.grid(axis="both")
ax.set_axisbelow(True)  # By default the axis is over the content.

#### Single agents

In [ ]:
for agent in env.agents:
    fig = plt.figure(layout="constrained")
    fig.canvas.header_visible = False
    ax = fig.subplots()

    for exp_dir in exps_dir:
        ratios_forwarded = (
            data[exp_dir][agent]["processed_forwarded_reqs"]
            / data[exp_dir][agent]["processed_reqs"]
        )

        ax.plot(ratios_forwarded, label=exp_dir.name)

    ax.set_title(f"Average processed forwarded requests per episode ({agent = })")

    ax.set_ylabel("Requests")
    ax.yaxis.set_major_formatter(ticker.PercentFormatter(1.0))
    ax.set_ylim(0, 1)

    ax.set_xlabel("Iteration")

    ax.legend(loc="lower center")
    ax.grid(axis="both")
    ax.set_axisbelow(True)  # By default the axis is over the content.